<a href="https://colab.research.google.com/github/K-107/dki/blob/main/PHR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HE_sbp: 수축기
HE_dbp: 이완기
HE_ht: 키
HE_wt: 몸무게
HE_glu: 공복혈당
HE_HB: 혈색소

age: 나이 Y

In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_columns', None)

df = pd.read_csv('/content/drive/MyDrive/PHR/PHR_20082017_raw.csv')
df = df[['age','HE_ht','HE_wt','HE_sbp','HE_dbp','HE_glu','HE_HB']].dropna().reset_index()
df.drop('index',axis=1,inplace=True)
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (155) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,age,HE_ht,HE_wt,HE_sbp,HE_dbp,HE_glu,HE_HB
0,51.0,168.0,57.6,110.0,80.0,89.0,15.0
1,46.0,156.4,73.1,114.0,79.0,98.0,11.9
2,25.0,157.9,50.7,99.0,69.0,87.0,13.2
3,54.0,173.2,89.6,121.0,83.0,99.0,15.1
4,49.0,159.2,67.0,111.0,81.0,90.0,11.2
...,...,...,...,...,...,...,...
65583,46.0,146.9,69.0,131.0,84.0,92.0,13.7
65584,22.0,160.1,40.2,109.0,73.0,88.0,13.4
65585,18.0,175.2,85.4,111.0,59.0,99.0,16.9
65586,77.0,158.2,60.3,126.0,80.0,88.0,16.2


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65588 entries, 0 to 65587
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     65588 non-null  float64
 1   HE_ht   65588 non-null  float64
 2   HE_wt   65588 non-null  float64
 3   HE_sbp  65588 non-null  float64
 4   HE_dbp  65588 non-null  float64
 5   HE_glu  65588 non-null  float64
 6   HE_HB   65588 non-null  float64
dtypes: float64(7)
memory usage: 3.5 MB


In [ ]:
age_100 = []
for i in range(100):
  age_100.append(i)

In [4]:
set(age_100) - set(sorted(df.age.unique()))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 94, 96, 98, 99}

In [5]:
df = df[(df.age>=9) & (df.age<=93)]
df

,age,HE_ht,HE_wt,HE_sbp,HE_dbp,HE_glu,HE_HB
0,51.0,168.0,57.6,110.0,80.0,89.0,15.0
1,46.0,156.4,73.1,114.0,79.0,98.0,11.9
2,25.0,157.9,50.7,99.0,69.0,87.0,13.2
3,54.0,173.2,89.6,121.0,83.0,99.0,15.1
4,49.0,159.2,67.0,111.0,81.0,90.0,11.2
...,...,...,...,...,...,...,...
65583,46.0,146.9,69.0,131.0,84.0,92.0,13.7
65584,22.0,160.1,40.2,109.0,73.0,88.0,13.4
65585,18.0,175.2,85.4,111.0,59.0,99.0,16.9
65586,77.0,158.2,60.3,126.0,80.0,88.0,16.2


In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled = scaler.fit_transform(df.drop('age', axis=1))
scaled

array([[0.69084423, 0.27211394, 0.23121387, 0.52287582, 0.07679466,
        0.67272727],
       [0.5529132 , 0.38830585, 0.25433526, 0.51633987, 0.0918197 ,
        0.48484848],
       [0.57074911, 0.22038981, 0.16763006, 0.45098039, 0.07345576,
        0.56363636],
       ...,
       [0.7764566 , 0.48050975, 0.23699422, 0.38562092, 0.09348915,
        0.78787879],
       [0.57431629, 0.29235382, 0.32369942, 0.52287582, 0.07512521,
        0.74545455],
       [0.52913199, 0.32683658, 0.3699422 , 0.49019608, 0.09015025,
        0.54545455]])

In [7]:
from sklearn.model_selection import train_test_split

X = scaled
y = df.age

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

LGBM = LGBMRegressor(n_estimators=200).fit(x_train,y_train)
pred = LGBM.predict(x_test)
mse = mean_squared_error(y_test, pred)
mae = mean_absolute_error(y_test, pred)
r2 = r2_score(y_test, pred)
print('MSE:  ', np.round(mse,3))
print('MAE: ', np.round(mae,3))
print('결정계수 R2: ', np.round(r2,3))

MSE:   185.387
MAE:  10.338
결정계수 R2:  0.509


In [16]:
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow as tf


model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(6,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1))

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae'])

model.fit(x_train,
          y_train,
          epochs=10,
          batch_size=32,
          )

results = model.evaluate(x_test, y_test)

Epoch 1/10
1640/1640 [==============================] - 7s 4ms/step - loss: 343.6585 - mae: 14.6380
Epoch 2/10
1640/1640 [==============================] - 6s 4ms/step - loss: 239.5452 - mae: 12.3947
Epoch 3/10
1640/1640 [==============================] - 6s 4ms/step - loss: 216.7582 - mae: 11.6124
Epoch 4/10
1640/1640 [==============================] - 6s 4ms/step - loss: 207.1397 - mae: 11.2302
Epoch 5/10
1640/1640 [==============================] - 6s 4ms/step - loss: 203.7162 - mae: 11.0980
Epoch 6/10
1640/1640 [==============================] - 6s 4ms/step - loss: 201.6098 - mae: 11.0124
Epoch 7/10
1640/1640 [==============================] - 6s 4ms/step - loss: 199.9581 - mae: 10.9461
Epoch 8/10
1640/1640 [==============================] - 6s 4ms/step - loss: 198.2655 - mae: 10.8687
Epoch 9/10
1640/1640 [==============================] - 6s 4ms/step - loss: 197.7678 - mae: 10.8447
Epoch 10/10
410/410 [==============================] - 1s 3ms/step - loss: 201.3208 - mae: 10.7907


In [17]:
results

[201.32083129882812, 10.790692329406738]

In [18]:
y_test

19851    40.0
3787     57.0
29774    24.0
45880    20.0
38451    64.0
         ... 
43752    73.0
58625    37.0
8899     16.0
55980    47.0
17354    11.0
Name: age, Length: 13118, dtype: float64

In [19]:
model.predict(x_test).reshape(-1)

array([46.258163, 40.44744 , 25.195845, ..., 29.69265 , 43.29418 ,
       61.782883], dtype=float32)

In [31]:
scaler.transform(t2)

array([[0.58858502, 0.36731634, 0.22543353, 0.50980392, 0.08347245,
        0.55757576],
       [0.46373365, 0.20689655, 0.1734104 , 0.47712418, 0.08180301,
        0.64242424],
       [0.81093936, 0.33958021, 0.24855491, 0.45751634, 0.06176962,
        0.63636364],
       ...,
       [0.84185493, 0.33733133, 0.26011561, 0.49019608, 0.0851419 ,
        0.70909091],
       [0.59690844, 0.26236882, 0.21965318, 0.45751634, 0.09015025,
        0.54545455],
       [0.4351962 , 0.25637181, 0.23699422, 0.45751634, 0.09682805,
        0.55757576]])

In [32]:
params = [[180.6, 72.8, 118, 79, 92, 14.5]]
params = scaler.transform(params)
model.predict(params)

array([[33.535954]], dtype=float32)

In [25]:
df_test = pd.DataFrame(data=y_test)
df_test['pred_age'] = model.predict(x_test).reshape(-1)
df_test

,age,pred_age
19851,40.0,46.258163
3787,57.0,40.447441
29774,24.0,25.195845
45880,20.0,35.796192
38451,64.0,38.238758
...,...,...
43752,73.0,49.299782
58625,37.0,44.352314
8899,16.0,29.692650
55980,47.0,43.294182


In [33]:
model.save('/content/drive/MyDrive/PHR/PHR.pb', save_format='tf')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/PHR/PHR.pb/assets
